In [43]:
# !pip install ultralytics
# !pip install roboflow

In [44]:
from datasets import load_dataset
from PIL import Image
import os
import json
from tqdm import tqdm

# Load dataset dari Hugging Face
train_dataset = load_dataset("naver-clova-ix/cord-v2", split="train")
test_dataset = load_dataset("naver-clova-ix/cord-v2", split="test")
val_dataset = load_dataset("naver-clova-ix/cord-v2", split="validation")

# Label map
label_map = {'table': 0, 'not_table': 1}

base_dir = "/kaggle/working/yolo_dataset"
for split in ['train', 'val']:
    os.makedirs(f"{base_dir}/images/{split}", exist_ok=True)
    os.makedirs(f"{base_dir}/labels/{split}", exist_ok=True)

# Fungsi konversi quadrilateral ke YOLO format
def quad_to_yolo(quad, img_w, img_h):
    x_coords = [quad[f'x{i}'] for i in range(1, 5)]
    y_coords = [quad[f'y{i}'] for i in range(1, 5)]
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)

    x_center = (x_min + x_max) / 2.0 / img_w
    y_center = (y_min + y_max) / 2.0 / img_h
    width = (x_max - x_min) / img_w
    height = (y_max - y_min) / img_h

    return x_center, y_center, width, height

def save_yolo_data(dataset, split):
    image_dir = os.path.join(base_dir, 'images', split)
    label_dir = os.path.join(base_dir, 'labels', split)

    for i, example in tqdm(enumerate(dataset), total=len(dataset), desc=f"Converting {split}"):
        parsed = json.loads(example['ground_truth'])
        gt_parse = parsed.get('gt_parse', {})
        image = example['image']
        img_w, img_h = image.size

        img_path = os.path.join(image_dir, f"{split}_{i:06d}.jpg")
        label_path = os.path.join(label_dir, f"{split}_{i:06d}.txt")
        image.save(img_path, format="JPEG")

        all_x, all_y = [], []

        # Kumpulkan semua koordinat quad dari item_name, quantity, price
        for item in gt_parse.get('menu', []):
            if not isinstance(item, dict):
                continue  # skip item yang berupa string/null

            item_name = item.get('nm', '')
            if isinstance(item_name, list):
                item_name = item_name[1] if len(item_name) > 1 else item_name[0] if item_name else ''
            item_name = str(item_name).strip().lower()

            quantity = str(item.get('cnt', '')).strip().lower()
            price = str(item.get('price', '')).strip().lower()

            for line in parsed.get('valid_line', []):
                words = line.get('words', [])
                text = ' '.join([w.get('text', '') for w in words]).strip().lower()

                if text in [item_name, quantity, price]:
                    for w in words:
                        if 'quad' in w:
                            for j in range(1, 5):
                                all_x.append(w['quad'][f'x{j}'])
                                all_y.append(w['quad'][f'y{j}'])

        # Jika ada koordinat, gabungkan menjadi satu bbox besar
        if all_x and all_y:
            merged_quad = {
                'x1': min(all_x), 'y1': min(all_y),
                'x2': max(all_x), 'y2': min(all_y),
                'x3': max(all_x), 'y3': max(all_y),
                'x4': min(all_x), 'y4': max(all_y),
            }

            x_c, y_c, w, h = quad_to_yolo(merged_quad, img_w, img_h)
            yolo_line = f"{label_map['table']} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}"

            with open(label_path, "w") as f:
                f.write(yolo_line)
        else:
            open(label_path, "w").close()

save_yolo_data(train_dataset, "train")
save_yolo_data(test_dataset, "train")
save_yolo_data(val_dataset, "val")

Converting val: 100%|██████████| 100/100 [00:05<00:00, 18.62it/s]


In [45]:
import os
from collections import Counter

label_dir = '/kaggle/working/yolo_dataset/labels/train'
label_counts = Counter()

for label_file in os.listdir(label_dir):
    if label_file.endswith(".txt"):
        with open(os.path.join(label_dir, label_file), 'r') as f:
            for line in f:
                if line.strip():  # skip baris kosong
                    class_id = int(line.split()[0])
                    label_counts[class_id] += 1

print("Total label yang ditemukan:")
for cls_id in sorted(label_counts):
    print(f"Class {cls_id}: {label_counts[cls_id]} objek")

total = sum(label_counts.values())
print(f"\n Total semua label: {total}")

Total label yang ditemukan:
Class 0: 2332 objek

 Total semua label: 2332


In [46]:
import shutil
from roboflow import Roboflow

rf = Roboflow(api_key="QYJz4fQRw4AUwlR6v8Wm")
project_receipt = rf.workspace("apple-academy").project("splivu")
version_receipt = project_receipt.version(1)
dataset_receipt = version_receipt.download("yolov8")

project_receipt2 = rf.workspace("wideai").project("r3c3ipt")
version_receipt2 = project_receipt2.version(6)
dataset_receipt2 = version_receipt2.download("yolov8")

project_receipt3 = rf.workspace("personal2").project("uhfhlsw-y6nak")
version_receipt3 = project_receipt3.version(1)
dataset_receipt3 = version_receipt3.download("yolov8")
                
project_receipt4 = rf.workspace("testing-gsoi0").project("sds-xusqb")
version_receipt4 = project_receipt4.version(1)
dataset_receipt4 = version_receipt4.download("yolov8")
                

print("Dataset: 'invoice'")
project_invoice = rf.workspace("helmetproject-vmo6o").project("invoice-pr8ex")
version_invoice = project_invoice.version(1)
dataset_invoice = version_invoice.download("yolov8")  # -> contoh: /kaggle/working/InvoiceDetection-2

project_invoice2 = rf.workspace("company-inc").project("invoicedetection-jqv45")
version_invoice2 = project_invoice2.version(2)
dataset_invoice2 = version_invoice2.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
Dataset: 'invoice'
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...


**MERGE BBOX AND RELABEL DATASET RECEIPT 2&3**

In [47]:
import os
import shutil
import yaml
from PIL import Image
from tqdm import tqdm

def merge_bboxes(bboxes):
    x_min = min(b[0] for b in bboxes)
    y_min = min(b[1] for b in bboxes)
    x_max = max(b[2] for b in bboxes)
    y_max = max(b[3] for b in bboxes)
    return [x_min, y_min, x_max, y_max]

# copy gambar, gabungkan bbox, simpan label baru
def process_dataset(dataset_dir, output_dir):
    data_yaml_path = os.path.join(dataset_dir, "data.yaml")
    with open(data_yaml_path, "r") as f:
        data_yaml_content = yaml.safe_load(f)

    class_id_to_name = {i: name for i, name in enumerate(data_yaml_content['names'])}

    target_labels = ["Item", "Price", "Quantity", "Sub Price"]

    for split in ['train', 'valid']:
        images_dir = os.path.join(dataset_dir, split, "images")
        labels_dir = os.path.join(dataset_dir, split, "labels")

        output_images_dir = os.path.join(output_dir, "images", split)
        output_labels_dir = os.path.join(output_dir, "labels", split)
        os.makedirs(output_images_dir, exist_ok=True)
        os.makedirs(output_labels_dir, exist_ok=True)

        for label_file in tqdm(os.listdir(labels_dir), desc=f"Processing {split} data"):
            if not label_file.endswith(".txt"):
                continue

            label_path = os.path.join(labels_dir, label_file)
            image_filename = label_file.replace(".txt", ".jpg")
            image_path = os.path.join(images_dir, image_filename)
            output_image_path = os.path.join(output_images_dir, image_filename)
            output_label_path = os.path.join(output_labels_dir, label_file)

            shutil.copy(image_path, output_image_path)

            with open(label_path, "r") as f:
                lines = f.readlines()

            bboxes = []
            image = Image.open(image_path)
            img_w, img_h = image.size

            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                class_id = int(parts[0])
                x_center = float(parts[1])
                y_center = float(parts[2])
                width = float(parts[3])
                height = float(parts[4])

                # Konversi bbox dari format YOLO ke koordinat absolut
                x_min = (x_center - width / 2) * img_w
                y_min = (y_center - height / 2) * img_h
                x_max = (x_center + width / 2) * img_w
                y_max = (y_center + height / 2) * img_h

                label_name = class_id_to_name.get(class_id, None)

                if label_name in target_labels:
                    bboxes.append([x_min, y_min, x_max, y_max])
                # Abaikan label lainnya

            if bboxes:
                merged_bbox = merge_bboxes(bboxes)
                # Konversi kembali ke format YOLO (relatif)
                x_center = ((merged_bbox[0] + merged_bbox[2]) / 2) / img_w
                y_center = ((merged_bbox[1] + merged_bbox[3]) / 2) / img_h
                width = (merged_bbox[2] - merged_bbox[0]) / img_w
                height = (merged_bbox[3] - merged_bbox[1]) / img_h

                with open(output_label_path, "w") as f:
                    f.write(f"{label_map['table']} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
            else:
                # tidak ada bbox target = file label kosong
                open(output_label_path, "w").close()


In [48]:
# Direktori output
output_dir = "/kaggle/working/yolo_dataset"

# Proses kedua dataset
process_dataset("/kaggle/working/r3c3ipt-6", output_dir)
process_dataset("/kaggle/working/uhfhlsw-1", output_dir)

Processing valid data: 100%|██████████| 8/8 [00:00<00:00, 1683.11it/s]


In [49]:
label_dir = '/kaggle/working/yolo_dataset/labels/train'
label_counts = Counter()

for label_file in os.listdir(label_dir):
    if label_file.endswith(".txt"):
        with open(os.path.join(label_dir, label_file), 'r') as f:
            for line in f:
                if line.strip():  # skip baris kosong
                    class_id = int(line.split()[0])
                    label_counts[class_id] += 1

print("Total label yang ditemukan:")
for cls_id in sorted(label_counts):
    print(f"Class {cls_id}: {label_counts[cls_id]} objek")

total = sum(label_counts.values())
print(f"\n Total semua label: {total}")

Total label yang ditemukan:
Class 0: 2332 objek

 Total semua label: 2332


**MERGE BBOX AND RELABEL DATASET RECEIPT 4**

In [50]:
import os
import shutil
import yaml
from PIL import Image
from tqdm import tqdm

def merge_bboxes(bboxes):
    x_min = min(b[0] for b in bboxes)
    y_min = min(b[1] for b in bboxes)
    x_max = max(b[2] for b in bboxes)
    y_max = max(b[3] for b in bboxes)
    return [x_min, y_min, x_max, y_max]

def process_dataset(dataset_dir, output_dir):
    data_yaml_path = os.path.join(dataset_dir, "data.yaml")
    with open(data_yaml_path, "r") as f:
        data_yaml_content = yaml.safe_load(f)

    # Mapping class_id ke nama label berdasarkan data.yaml
    class_id_to_name = {i: name for i, name in enumerate(data_yaml_content['names'])}

    # Label yang ingin digabungkan
    target_labels = [
        "==============================",
        "Roboflow is an end-to-end computer vision platform that helps you",
        "This dataset was exported via roboflow.com on November 24, 2024 at 1:47 AM GMT", 
        "r3c3ipt - v6 2023-08-25 10:58am" 
    ]

    for split in ['train', 'valid']:
        images_dir = os.path.join(dataset_dir, split, "images")
        labels_dir = os.path.join(dataset_dir, split, "labels")

        output_images_dir = os.path.join(output_dir, "images", split)
        output_labels_dir = os.path.join(output_dir, "labels", split)
        os.makedirs(output_images_dir, exist_ok=True)
        os.makedirs(output_labels_dir, exist_ok=True)

        for label_file in tqdm(os.listdir(labels_dir), desc=f"Processing {split} data"):
            if not label_file.endswith(".txt"):
                continue

            label_path = os.path.join(labels_dir, label_file)
            image_filename = label_file.replace(".txt", ".jpg")
            image_path = os.path.join(images_dir, image_filename)
            output_image_path = os.path.join(output_images_dir, image_filename)
            output_label_path = os.path.join(output_labels_dir, label_file)

            shutil.copy(image_path, output_image_path)

            with open(label_path, "r") as f:
                lines = f.readlines()

            bboxes = []
            image = Image.open(image_path)
            img_w, img_h = image.size

            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                class_id = int(parts[0])
                x_center = float(parts[1])
                y_center = float(parts[2])
                width = float(parts[3])
                height = float(parts[4])

                # Konversi bbox dari format YOLO ke koordinat absolut
                x_min = (x_center - width / 2) * img_w
                y_min = (y_center - height / 2) * img_h
                x_max = (x_center + width / 2) * img_w
                y_max = (y_center + height / 2) * img_h

                label_name = class_id_to_name.get(class_id, None)

                if label_name in target_labels:
                    bboxes.append([x_min, y_min, x_max, y_max])

            if bboxes:
                merged_bbox = merge_bboxes(bboxes)
                # Konversi kembali ke format YOLO (relatif)
                x_center = ((merged_bbox[0] + merged_bbox[2]) / 2) / img_w
                y_center = ((merged_bbox[1] + merged_bbox[3]) / 2) / img_h
                width = (merged_bbox[2] - merged_bbox[0]) / img_w
                height = (merged_bbox[3] - merged_bbox[1]) / img_h

                with open(output_label_path, "w") as f:
                    f.write(f"{label_map['table']} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
            else:
                open(output_label_path, "w").close()

# Proses dataset
process_dataset(dataset_receipt4.location, output_dir)


Processing valid data: 100%|██████████| 16/16 [00:00<00:00, 1814.93it/s]


In [51]:
label_dir = '/kaggle/working/yolo_dataset/labels/train'
label_counts = Counter()

for label_file in os.listdir(label_dir):
    if label_file.endswith(".txt"):
        with open(os.path.join(label_dir, label_file), 'r') as f:
            for line in f:
                if line.strip():  # skip baris kosong
                    class_id = int(line.split()[0])
                    label_counts[class_id] += 1

print("Total label yang ditemukan:")
for cls_id in sorted(label_counts):
    print(f"Class {cls_id}: {label_counts[cls_id]} objek")

total = sum(label_counts.values())
print(f"\n Total semua label: {total}")

Total label yang ditemukan:
Class 0: 2332 objek

 Total semua label: 2332


**RELABEL OTHER DATASET**

In [52]:
import yaml

def cek_label_names(data_yaml_path):
    with open(data_yaml_path, 'r') as f:
        data = yaml.safe_load(f)
    names = data.get('names', [])
    print("📋 Daftar Label:")
    for i, name in enumerate(names):
        print(f"  {i}: {name}")
    return names

data_yaml_invoice = os.path.join(dataset_invoice.location, "data.yaml")
label_invoice = cek_label_names(data_yaml_invoice)

data_yaml_invoice2 = os.path.join(dataset_invoice2.location, "data.yaml")
label_invoice2 = cek_label_names(data_yaml_invoice2)

data_yaml_receipt = os.path.join(dataset_receipt.location, "data.yaml")
label_receipt = cek_label_names(data_yaml_receipt)

📋 Daftar Label:
  0: -
  1: Invoice- NER detection - v1 2024-03-10 8-52am
📋 Daftar Label:
  0: CLIENT
  1: COMPANY
  2: CUSTOMER
  3: Customer
  4: CustomerName
  5: INVOICE
  6: PAYMENT
  7: TABLE
  8: TOTAL
📋 Daftar Label:
  0: receipt


In [53]:
def filter_labels_keep_only_class_0(label_dir, keep_class_id=0):
    kept_files = 0
    removed_lines = 0

    for file in os.listdir(label_dir):
        path = os.path.join(label_dir, file)
        if path.endswith('.txt') and os.path.isfile(path):
            with open(path, 'r') as f:
                lines = f.readlines()

            new_lines = [line for line in lines if line.strip().startswith(str(keep_class_id) + ' ')]
            removed = len(lines) - len(new_lines)

            if new_lines:
                with open(path, 'w') as f:
                    f.writelines(new_lines)
                kept_files += 1
                removed_lines += removed
            else:
                os.remove(path)  # Hapus file jika semua label dihapus

    print(f"✅ {kept_files} file disimpan dengan hanya label class {keep_class_id}.")
    print(f"🗑️ {removed_lines} baris label lain dibuang.")

filter_labels_keep_only_class_0(os.path.join(dataset_receipt.location, 'train', 'labels'))
filter_labels_keep_only_class_0(os.path.join(dataset_receipt.location, 'valid', 'labels'))

✅ 146 file disimpan dengan hanya label class 0.
🗑️ 0 baris label lain dibuang.
✅ 15 file disimpan dengan hanya label class 0.
🗑️ 0 baris label lain dibuang.


In [54]:
# Dataset InvoiceDetection hanya pakai label "-"
filter_labels_keep_only_class_0(os.path.join(dataset_invoice.location, 'train', 'labels'))
filter_labels_keep_only_class_0(os.path.join(dataset_invoice.location, 'valid', 'labels'))
filter_labels_keep_only_class_0(os.path.join(dataset_invoice2.location, 'train', 'labels'), 7)
filter_labels_keep_only_class_0(os.path.join(dataset_invoice2.location, 'valid', 'labels'), 7)

✅ 693 file disimpan dengan hanya label class 0.
🗑️ 0 baris label lain dibuang.
✅ 65 file disimpan dengan hanya label class 0.
🗑️ 0 baris label lain dibuang.
✅ 0 file disimpan dengan hanya label class 7.
🗑️ 0 baris label lain dibuang.
✅ 0 file disimpan dengan hanya label class 7.
🗑️ 0 baris label lain dibuang.


In [55]:
def relabel_class_0_to_target(label_dir, new_class_id=0):
    for file in os.listdir(label_dir):
        path = os.path.join(label_dir, file)
        if path.endswith('.txt') and os.path.isfile(path):
            with open(path, 'r') as f:
                lines = f.readlines()

            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if parts[0] == '0':
                    parts[0] = str(new_class_id)
                    new_lines.append(' '.join(parts))

            with open(path, 'w') as f:
                f.write('\n'.join(new_lines))

relabel_class_0_to_target(os.path.join(dataset_receipt.location, 'train', 'labels'), 0)
relabel_class_0_to_target(os.path.join(dataset_receipt.location, 'valid', 'labels'), 0)

In [56]:
relabel_class_0_to_target(os.path.join(dataset_invoice.location, 'train', 'labels'), 0)
relabel_class_0_to_target(os.path.join(dataset_invoice.location, 'valid', 'labels'), 0)
relabel_class_0_to_target(os.path.join(dataset_invoice2.location, 'train', 'labels'), 0)
relabel_class_0_to_target(os.path.join(dataset_invoice2.location, 'valid', 'labels'), 0)

In [57]:
# Initialize overall counters for each dataset type
total_receipt_labels = Counter()
total_invoice1_labels = Counter()
total_invoice2_labels = Counter()

print("Mulai menggabungkan dataset dan menghitung label...")

for split in ['train', 'valid']:
    # Initialize split-specific counters for each dataset type
    split_receipt_labels = Counter()
    split_invoice1_labels = Counter()
    split_invoice2_labels = Counter()

    for subfolder in ['images', 'labels']:
        src1 = os.path.join(dataset_receipt.location, split, subfolder)
        src2 = os.path.join(dataset_invoice.location, split, subfolder)
        src3 = os.path.join(dataset_invoice2.location, split, subfolder)
        dst = os.path.join(base_dir, subfolder, split)

        os.makedirs(dst, exist_ok=True)

        for i, src in enumerate([src1, src2, src3], start=1):
            if os.path.exists(src):
                files = [f for f in os.listdir(src) if os.path.isfile(os.path.join(src, f))]
                print(f"Menyalin {len(files)} file dari sumber {i} ({split}/{subfolder})")

                for f in files:
                    shutil.copy(os.path.join(src, f), os.path.join(dst, f))

                    # If it's a labels subfolder and a .txt file, count the labels
                    if subfolder == 'labels' and f.endswith(".txt"):
                        label_file_path = os.path.join(src, f)
                        with open(label_file_path, 'r') as label_f:
                            for line in label_f:
                                if line.strip():  # Skip empty lines
                                    class_id = int(line.split()[0])
                                    if i == 1: # dataset_receipt
                                        split_receipt_labels[class_id] += 1
                                        total_receipt_labels[class_id] += 1
                                    elif i == 2: # dataset_invoice (invoice 1)
                                        split_invoice1_labels[class_id] += 1
                                        total_invoice1_labels[class_id] += 1
                                    elif i == 3: # dataset_invoice2 (invoice 2)
                                        split_invoice2_labels[class_id] += 1
                                        total_invoice2_labels[class_id] += 1
            else:
                print(f"Folder tidak ditemukan: {src}")

    # Print label counts for the current split
    print("\n---")
    print(f"Jumlah Label per Class untuk Split '{split}':")
    print("Dataset Receipt:")
    if split_receipt_labels:
        for cls_id in sorted(split_receipt_labels):
            print(f"  Class {cls_id}: {split_receipt_labels[cls_id]} objek")
    else:
        print("  Tidak ada label ditemukan.")

    print("\nDataset Invoice 1:")
    if split_invoice1_labels:
        for cls_id in sorted(split_invoice1_labels):
            print(f"  Class {cls_id}: {split_invoice1_labels[cls_id]} objek")
    else:
        print("  Tidak ada label ditemukan.")

    print("\nDataset Invoice 2:")
    if split_invoice2_labels:
        for cls_id in sorted(split_invoice2_labels):
            print(f"  Class {cls_id}: {split_invoice2_labels[cls_id]} objek")
    else:
        print("  Tidak ada label ditemukan.")
    print("---")


print("\nSelesai menggabungkan dataset dan menghitung label.")

Mulai menggabungkan dataset dan menghitung label...
Menyalin 146 file dari sumber 1 (train/images)
Menyalin 774 file dari sumber 2 (train/images)
Menyalin 5310 file dari sumber 3 (train/images)
Menyalin 146 file dari sumber 1 (train/labels)
Menyalin 693 file dari sumber 2 (train/labels)
Menyalin 0 file dari sumber 3 (train/labels)

---
Jumlah Label per Class untuk Split 'train':
Dataset Receipt:
  Class 0: 146 objek

Dataset Invoice 1:
  Class 0: 717 objek

Dataset Invoice 2:
  Tidak ada label ditemukan.
---
Menyalin 15 file dari sumber 1 (valid/images)
Menyalin 73 file dari sumber 2 (valid/images)
Menyalin 324 file dari sumber 3 (valid/images)
Menyalin 15 file dari sumber 1 (valid/labels)
Menyalin 65 file dari sumber 2 (valid/labels)
Menyalin 0 file dari sumber 3 (valid/labels)

---
Jumlah Label per Class untuk Split 'valid':
Dataset Receipt:
  Class 0: 15 objek

Dataset Invoice 1:
  Class 0: 71 objek

Dataset Invoice 2:
  Tidak ada label ditemukan.
---

Selesai menggabungkan dataset 

In [58]:
import yaml
import os

dataset_yaml = {
    'path': '/kaggle/working/yolo_dataset',
    'train': 'images/train',
    'val': 'images/val',
    'nc': 2,
    'names': {
        0: 'table',
        1: 'not_table'
    }
}

os.makedirs('/kaggle/working/yolo_dataset', exist_ok=True)

# Simpan dataset.yaml
with open('/kaggle/working/yolo_dataset/dataset.yaml', 'w') as f:
    yaml.dump(dataset_yaml, f, sort_keys=False)

In [59]:
label_dir = '/kaggle/working/yolo_dataset/labels/train' 
label_counts = Counter()

for label_file in os.listdir(label_dir):
    if label_file.endswith(".txt"):
        with open(os.path.join(label_dir, label_file), 'r') as f:
            for line in f:
                if line.strip():  # skip baris kosong
                    class_id = int(line.split()[0])
                    label_counts[class_id] += 1

print("Total label yang ditemukan:")
for cls_id in sorted(label_counts):
    print(f"Class {cls_id}: {label_counts[cls_id]} objek")

total = sum(label_counts.values())
print(f"\nTotal semua label: {total}")

Total label yang ditemukan:
Class 0: 2335 objek

Total semua label: 2335


In [60]:
img_dir = '/kaggle/working/yolo_dataset/images/train'
label_dir = '/kaggle/working/yolo_dataset/labels/train'

img_names = set(f.rsplit('.', 1)[0] for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png')))
label_names = set(f.rsplit('.', 1)[0] for f in os.listdir(label_dir) if f.endswith('.txt'))

missing_labels = img_names - label_names
print(f"[1] Menghapus {len(missing_labels)} gambar tanpa label...")

for name in missing_labels:
    for ext in ['.jpg', '.png']:
        img_path = os.path.join(img_dir, name + ext)
        if os.path.exists(img_path):
            os.remove(img_path)

segment_files = []

for label_file in os.listdir(label_dir):
    if label_file.endswith(".txt"):
        with open(os.path.join(label_dir, label_file), 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) > 5:
                    segment_files.append(label_file)
                    break  # cukup satu baris untuk tahu ini segmentasi

print(f"[2] Menghapus {len(segment_files)} gambar & label dengan segmentasi...")

for f in segment_files:
    name = f.rsplit('.', 1)[0]
    
    # Hapus label
    label_path = os.path.join(label_dir, f)
    if os.path.exists(label_path):
        os.remove(label_path)
    
    # Hapus gambar pasangannya (kalau ada)
    for ext in ['.jpg', '.png']:
        img_path = os.path.join(img_dir, name + ext)
        if os.path.exists(img_path):
            os.remove(img_path)

print("\nSelesai membersihkan dataset.")

[1] Menghapus 4415 gambar tanpa label...
[2] Menghapus 3 gambar & label dengan segmentasi...

Selesai membersihkan dataset.


In [61]:
img_dir = '/kaggle/working/yolo_dataset/images/train'
label_dir = '/kaggle/working/yolo_dataset/labels/train'

img_files = set(f.rsplit('.', 1)[0] for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png')))
label_files = set(f.rsplit('.', 1)[0] for f in os.listdir(label_dir) if f.endswith('.txt'))

missing_labels = img_files - label_files
print(f"{len(missing_labels)} gambar tidak memiliki label.")

0 gambar tidak memiliki label.


In [62]:
import os

label_dir = '/kaggle/working/yolo_dataset/labels/train'
segment_files = []

for label_file in os.listdir(label_dir):
    if label_file.endswith(".txt"):
        with open(os.path.join(label_dir, label_file), 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) > 5:
                    segment_files.append(label_file)
                    break  # cukup satu baris aja untuk memastikan file ini segmentasi

print(f"Jumlah file label yang mengandung segmentasi: {len(segment_files)}")
print("Contoh file segmentasi:", segment_files)

Jumlah file label yang mengandung segmentasi: 0
Contoh file segmentasi: []


In [63]:
label_dir = '/kaggle/working/yolo_dataset/labels/train'
obj_count_per_image = []

for file in os.listdir(label_dir):
    if file.endswith('.txt'):
        with open(os.path.join(label_dir, file), 'r') as f:
            lines = [line for line in f if line.strip()]
            obj_count_per_image.append(len(lines))

counter = Counter(obj_count_per_image)

print("Distribusi jumlah objek class 0 per gambar:")
for k in sorted(counter):
    print(f"{k} objek: {counter[k]} gambar")

print(f"\nTotal gambar dengan objek: {len(obj_count_per_image)}")
print(f"Total objek class 0: {sum(obj_count_per_image)}")

Distribusi jumlah objek class 0 per gambar:
0 objek: 1319 gambar
1 objek: 2287 gambar
2 objek: 18 gambar
3 objek: 3 gambar

Total gambar dengan objek: 3627
Total objek class 0: 2332


In [64]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

results = model.train(
    data = '/kaggle/working/yolo_dataset/dataset.yaml',
    batch = 24,
    epochs = 35,
    device = 0,
    lr0 = 3e-4,
    optimizer='AdamW',
    imgsz = 640,
    save_period = 10
)

Ultralytics 8.3.158 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=35, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0003, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12

train: Scanning /kaggle/working/yolo_dataset/labels/train.cache... 3627 images, 1319 backgrounds, 0 corrupt: 100%|██████████| 3627/3627 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 445.2±121.6 MB/s, size: 209.4 KB)


val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 100 images, 46 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]


...

      35/35      5.75G     0.4962      0.495     0.9041          2        640: 100%|██████████| 152/152 [00:34<00:00,  4.36it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  6.12it/s]

                   all        100         54      0.988      0.926      0.988      0.893



35 epochs completed in 0.353 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.158 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.30it/s]
invalid value encountered in less
invalid value encountered in less


                   all        100         54      0.988      0.926      0.988      0.891
                 table         54         54      0.988      0.926      0.988      0.891
Speed: 0.1ms preprocess, 2.0ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train3


In [65]:
import shutil

# Path folder yang ingin dikompres (misalnya hasil training YOLOv8)
folder_path = "/kaggle/working/runs/detect/train"
zip_path = "/kaggle/working/train_results.zip"

# Buat zip dari folder
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', folder_path)

print("Folder berhasil dikompres ke:", zip_path)

Folder berhasil dikompres ke: /kaggle/working/train_results.zip


In [ ]:
from PIL import Image
from IPython.display import display

img = Image.open("/kaggle/working/runs/detect/train/results.png")
display(img)

In [ ]:
from ultralytics import YOLO

model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")

In [ ]:
import cv2
import numpy as np

dataset = load_dataset("naver-clova-ix/cord-v2", split="train")
# image_path = dataset[12]['image']  # Sudah dalam bentuk PIL.Image
image_path = Image.open("/kaggle/input/contoh/contoh-1.jpeg")
image_np = cv2.cvtColor(np.array(image_path), cv2.COLOR_RGB2BGR)
image_resized = cv2.resize(image_np, (600, 600))

results = model(image_resized)  # Ganti dengan path gambar kamu


# Tampilkan hasilnya
results[0].show()

In [ ]:
for result in results:
    boxes = result.boxes  # bounding boxes
    probs = result.probs  # klasifikasi probabilitas (jika ada)
    print(boxes.xyxy)     # koordinat [x1, y1, x2, y2]
    print(boxes.conf)     # confidence score
    print(boxes.cls)      # class index

In [ ]:
for box in results[0].boxes:
    print(box.xyxy, box.conf, box.cls)